<a href="https://colab.research.google.com/github/shanaoka50/MyTrade/blob/master/SimpleTrader_0_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SimpleTrader 試作1号 改
###SimpleTrader_0_2よりの変更点
<li>モデルのパラメータをSave-Load可能にする</li>
<li>報酬の考え方をさらに修正</li>
<li>インプットデータの抽出を週単位から時間単位までできるようにする</li>
<li>エピソード毎(あるいはエピソード内の週ごと)にトレード回数、最大損失、最大利益、平均、偏差を取得する</li>

##実現したいこと

<li>Python のMetatrader5モジュールを試す [参考URL](https://www.mql5.com/en/docs/integration/python_metatrader5)</li>
<li>テスト期間を任意に設定できるようにする。(事前読み込みN時間、N週、Nか月、トレーニングN時間、N週、Nか月)</li>
<li>トレード実績を週単位で集計するisocalendar(year,weeknum,_),tradecount,average,sddev</li>
<li>Google Driveをマウントして、直接ファイル入出力をする</li>
  
  


##参考URL
[第15回　CartPole課題で深層強化学習DQNを実装](https://book.mynavi.jp/manatee/detail/id=89831)

[SEQUENCE MODELS AND LONG-SHORT TERM MEMORY NETWORKS](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sequence-models-and-long-short-term-memory-networks)

##次のステップ
<li>複数の通貨ペア時間足を読み込む
<li>トレーニング期間の変更：例えば、2週間を読み込んでも、6か月を読<li>み込んでも、始めの1週間は何もしないで、次の週からトレードを開始する
<li>モデルパラメータの書き出し、読み込み
<li>トレーニング成果の検証：モデルのパラメータを読み込み、任意の期間のトレード実績を測定
  
 ##その次
<li>MetaTrader4/5との連携
<li>OWANDA APIとの連携    




##GoogleDriveのマウント

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##Dependency

In [0]:
import numpy as np
import pandas as pd
import random
import glob #ファイル名の正規化モジュール
from datetime import datetime,timedelta

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import sys #デバッグ用にプログラムを途中終了させる

##Class Environment

・エージェントに行動に応じた状態と次に取れる行動の選択肢を与える

・エージェントに報酬を与える

###__init__(self)
損益情報とポジション情報を初期化する。<br>

###makePriceData(self,symbol,period,filepath='')
MetaTraderのデモサイトから取得したpricedataを加工する。
取得データはCSV形式で<DATE(0)\><TIME(1)\><OPEN(2)\>	<HIGH(3)\>	<LOW(4)\>	<CLOSE(5)\>	<TICKVOL(6)\>	<VOL(7)\>	<SPREAD(8)\>となっているので、次のようにする。<br>

0_DateTime：取得データ(data[])のDate(data[0])とTime(data[1])を結合し、pythonのDateTime形式にして変換する。<br>
1_WeekNum：上記のDateTimeがisoCalendarの何週目か。datetime.isocalendar()[1]を取得する。<br>
2_Weekday：上記のDateTimeがisoCalendarの何曜日か。datetime.isoweekday()を取得する。<br>
3_Open：data[2]のデータを格納する。<br>
4_High：data[3]のデータを格納する。<br>
5_Low：data[4]のデータを格納する。<br>
6_Close：data[5]のデータを格納する。<br>
7_Tickvol：data[6]/100000を格納する。<br>

###getInitialPriceData(self, pricedata_week)
1週間分のpricedataをニューラルネットのインプットデータに加工する。ポジション情報はダミーとして0で埋める。<br>

0_ProfitLoss：その時点の損益。ここでは0を入れる<br>
1_LongPosition：ロングポジションの有無。ここでは0(False)を入れる<br>
2_ShortPosition：ショーとポジションの有無。ここでは0(False)を入れる<br>
3_WeekDay：現在の曜日。月曜日=1/7、火曜日=2/7、…日曜日=7/7<br>
4_Countdown：毎週月曜日の0:00足を1、金曜日の22:00足を0となるようにする。金曜日の23:00足はマイナスの値となる。<br>
5_Close-Open：その足の「終値」-「始値」。ただし、EURUSD、USDGBP、EURGBPは×100、USDJPY、EURJPY、GBPJPYは×1とする。<br>
6_High-Low：その足の「高値」-「安値」。ただし、EURUSD、USDGBP、EURGBPは×100、USDJPY、EURJPY、GBPJPYは×1とする。<br>
7_Tickvol：1/100000とする。

###getAvailableAction(self,(plofit_loss, long_position, short_position),CountDown)
現在のポジションの状況から、可能な行動をマスク配列で返す。<br>

<b>取引可能時間帯：COUNTDOWN >0 </b>

<li>パターン1：ポジションなし(long_position = False, short_position = False)<br>
  新規ポジションエントリー(long,shot)と何もしないが選択可能。<br>
 action_mask[0] = [a0=True, a1=True, a2=True, a3=False, a4=False, a5=False]<br>
<li>パターン2：ロングポジション(long_position = True, short_position = False)<br>
  ポジションクローズと、クローズ＆追加エントリ、何もしないが選択可能。<br>
 action_mask[1] = [a0=True, a1=False, a2=False, a3=True, a4=True, a5=True]<br>
<li>パターン3：ショートポジション(long_position = False, short_position = True)<br>
  ポジションクローズと、クローズ＆追加エントリ、何もしないが選択可能。<br>
 action_mask[2] =[a0=True, a1=False, a2=False, a3=True, a4=True, a5=True]<br>
<li>パターン4：両建て(long_position = True, short_position = True)<br>
  ポジションクローズ(全決済)が選択可能。両建ては禁止とするため、このパターンにはならない。<br>
 action_mask[3] =[a0=False, a1=False, a2=False, a3=True, a4=False, a5=False]<br>
  
  <b>取引可能時間外：COUNTDOWN \<=0 </b>
<li>パターン5：ポジションなし(long_position = False, short_position = False)<br>
  何もしないが選択可能。<br>
 action_mask[4] = [a0=True, a1=False, a2=False, a3=False, a4=False, a5=False]<br>
<li>パターン6：ロングポジション(long_position = True, short_position = False)<br>
  ポジションクローズが選択可能。<br>
 action_mask[5] = [a0=False, a1=False, a2=False, a3=True, a4=False, a5=False]<br>
<li>パターン7：ショートポジション(long_position = False, short_position = True)<br>
  ポジションクローズが選択可能。<br>
 action_mask[6] = [a0=False, a1=False, a2=False, a3=True, a4=False, a5=False]<br>
<li>パターン8：両建て(long_position = True, short_position = True)<br>
  ポジションクローズ(全決済)が選択可能。両建ては禁止とするため、このパターンにはならない。<br>
 action_mask[7] =[a0=False, a1=False, a2=False, a3=True, a4=False, a5=False]<br>


In [0]:
#@title class Environment { output-height: 100 }
#@markdown コメント <b>HTMLタグも利用可能</b>
POSITION_OPEN_PRICE = 0.0 #@param {type:"number"}
PROFIT_LOSS = 0.0 #@param {type:"number"}
TOTAL_PROFIT_LOSS = 0.0 #@param {type:"number"}
REWARD = 0.0 #@param {type:"number"}
LONG_POSITION = False #@param {type:"boolean"}
SHORT_POSITION = False #@param {type:"boolean"}
# スプレッド(1.0pips相当)
MARGIN = 0.01 #@param {type:"number"}

class Environment:
  
  def __init__(self,sym,prd):
    self.symbol = sym
    self.period = prd
    self.clearAccountState()
    return
  
  def clearAccountState(self):
    self.position_open_price = POSITION_OPEN_PRICE
    self.profit_loss = PROFIT_LOSS
    # self.total_profit_loss = TOTAL_PROFIT_LOSS
    self.total_profit_loss = np.empty(0)
    self.reward = REWARD
    self.long_position = LONG_POSITION
    self.short_position = SHORT_POSITION
    self.setLosscutLevel()
    return
  
  def setLosscutLevel(self):
    # Losscut,Takeprofitのレベルを時間足に対応して決める
    # 基準はH1の時に1.0、D1の時に2.0となるようにする
    # y = (1/23)x + 22/23
    if(self.period is 'M1'):
      self.period_value = (1/60)
    elif(self.period is 'M5'):
      self.period_value = (5/60)
    elif(self.period is 'M15'):
      self.period_value = (15/60)
    elif(self.period is 'M30'):
      self.period_value = (30/60)
    elif(self.period is 'H1'):
      self.period_value = 1
    elif(self.period is 'H4'):
      self.period_value = 4
    elif(self.period is 'H6'):
      self.period_value = 6
    elif(self.period is 'D1'):
      self.period_value = 24
    else:
      print('■■Period is not Implemented')
      self.period_value = 1
    
    self.losscut_level = (1/23)*self.period_value + (22/23)
    # print('## Losscut Level: %d' %(self.losscut_level))
    return
  
  def makePriceData(self,symbol,period,filepath=''):
    Delimiter='\t'
    #ファイルを読み込む
    self.symbol=symbol
    self.period=period
    self.filepath=filepath
    
    self.filename=glob.glob('%s%s_%s*.csv'%(filepath,symbol,period))
    # print(self.filename)
    if len(self.filename) == 0:
      #print('File not found.')
      sys.exit('Price data file not found.')
      
    # np.append()を使うと処理が重くなるのでpython listを使う
    self.price_list = [['DateTime','WeekNum','Weekday','Open','High','Low','Close','Tickvol']]
    
    for fname in self.filename:
      with open(fname, "r") as f:
        #1行目はヘッダ行
        line=f.readline()
        # self.price=np.array([['DateTime','WeekNum','Weekday','Open','High','Low','Close','Tickvol']])
        # print(self.price)
        line=f.readline()

        while line:
          #1行のデータを列ごとに分割する
          data=np.array(line.rstrip().split(Delimiter))
          
          #DateTimeの加工。取得データ(data[])のDate(data[0])とTime(data[1])を結合し、pythonのDateTime形式にして変換する。
          # dtDateTime=datetime.datetime.strptime('%s %s'%(data[0],data[1]), '%Y.%m.%d %H:%M:%S')
          dtDateTime=datetime.strptime('%s %s'%(data[0],data[1]), '%Y.%m.%d %H:%M:%S')
          
          # self.price = np.append(self.price,[[dtDateTime, dtDateTime.isocalendar()[1], dtDateTime.weekday(),*data[2:7]]]
          #                        ,0)
          self.price_list.extend([[dtDateTime, dtDateTime.isocalendar()[1], dtDateTime.isoweekday(),*data[2:7]]])
                                
          line=f.readline()
        # print(self.price[0:10,:])
    # 分足等複数ファイルを読み込んだ場合は、すべてを読み込んでからnp.array()する。
    self.price = np.array(self.price_list)
  
  def getInitialPriceData(self, pricedata_week):
    # pricedata_weekに損益情報とポジション情報(すべて0)を結合して返す
    # 今後は、複数のpricedata_week[]に対応できるようにする
    self.initial_data = np.empty((pricedata_week.shape[0],8),dtype=np.float)
    
    for i, data in enumerate(pricedata_week):
      #Countdownの加工。毎週月曜日の0:00足を1、金曜日の22:00足を0となるようにする。金曜日の23:00足はマイナスの値となる。
      dtStartTrade=(data[0] + timedelta(days=1-data[0].isoweekday())).replace(hour=0,minute=0)
      dtEndTrade=(data[0] + timedelta(days=5-data[0].isoweekday())).replace(hour=22,minute=0)

      #'Close-Open'、'High-Low'の加工。
      fCloseOpen = float(data[6])-float(data[3])
      fHighLow = float(data[4])-float(data[5])
      
      # ProfitLossの計算。ポジションがオープンの場合に計算する。
      # 100.0pips = 1 となるようにする。
      if(self.position_open_price > 0.0):
        if(bool(self.long_position) & bool(not self.short_position)):
          self.profit_loss = float(data[6]) - self.position_open_price
        elif(bool(not self.long_position) & bool(self.short_position)):
          self.profit_loss = self.position_open_price - float(data[6])
        else:
          print("■■entry position error■■")
          self.profit_loss = 0.0
       
        # EURUSDのような下5桁通貨は100倍する
        if (self.symbol is 'EURUSD') or (self.symbol is 'GBPUSD') or (self.symbol is 'EURGBP'):
          self.profit_loss = self.profit_loss*100
          fCloseOpen = fCloseOpen*100
          fHighLow = fHighLow*100
        
        # 損益からスプレッドを差し引く
        self.profit_loss -= MARGIN
      
      else:
        self.profit_loss = 0.0
  
      # ProfitLossが100.0pips(=1.000)を超えた場合はTakeProfitし
      # -100.0pips(=-1.000)を超えた場合はLossCutする。
      # Rewardを1もしくは-1とし、ポジションを解消する
      if( self.profit_loss > self.losscut_level):
        self.reward += 1.0 - MARGIN
        # self.total_profit_loss += (self.reward - MARGIN)
        self.total_profit_loss = np.append(self.total_profit_loss, self.reward)
        self.position_open_price = 0.0
        self.profit_loss = 0.0
        self.long_position = False
        self.short_position = False
        print('### TakeProfit Executed.')
        print(' Reward:{:.4f}, TotalProfitLoss:{:.4f}, PositionOpenPrice:{:.4f}, ProfitLoss:{:.4f}, LongPosition:{:.4f}, ShortPosition:{:.4f}'
             .format(self.reward, np.sum(self.total_profit_loss), self.position_open_price, self.profit_loss, self.long_position, self.short_position))
      elif( self.profit_loss < -1 * self.losscut_level):
        self.reward += -1.0 - MARGIN
        # self.total_profit_loss += (self.reward - MARGIN)
        self.total_profit_loss = np.append(self.total_profit_loss, self.reward)
        self.position_open_price = 0.0
        self.profit_loss = 0.0
        self.long_position = False
        self.short_position = False
        print('### LossCut Executed.')
        print(' Reward:{:.4f}, TotalProfitLoss:{:.4f}, PositionOpenPrice:{:.4f}, ProfitLoss:{:.4f}, LongPosition:{:.4f}, ShortPosition:{:.4f}'
             # .format(self.reward, self.total_profit_loss, self.position_open_price, self.profit_loss, self.long_position, self.short_position))
             .format(self.reward, np.sum(self.total_profit_loss), self.position_open_price, self.profit_loss, self.long_position, self.short_position))
          
      #self.initial_data[i] = [*np.zeros(3), float(data[2])/6,
      self.initial_data[i] = [self.profit_loss, self.long_position, self.short_position, float(data[2])/7,
                              (dtEndTrade-data[0]).total_seconds()/(dtEndTrade-dtStartTrade).total_seconds(),
                              fCloseOpen, fHighLow, float(data[7])/100000]
      
    return self.initial_data

  def getPriceData(self,year=None,weeknum=None,weekdaynum=None,hournum=None):
    #Price dataを年間の週、曜日、時間によって切り出す
    self.extract_price = self.price
    
    if(year is not None):
      self.extract_price = np.array([d for d in self.extract_price[1:] if d[0].year == year])
    if(weeknum is not None):
      self.extract_price = self.extract_price[self.extract_price[:,1] == weeknum]
    if(weekdaynum is not None):
      self.extract_price = self.extract_price[self.extract_price[:,2] == weekdaynum]
    if(hournum is not None):
      self.extract_price = np.array([d for d in self.extract_price if d[0].hour == hournum])
      
    # return self.price[self.price[:,1] == weeknum]
    # print('PriceData')
    # print(self.extract_price[0:10])
    return self.extract_price 

  def getAvailableAction(self,pl_, lp_, sp_, cd_):
    self.action_mask = np.empty(6, dtype='bool')
    
    #現在の状況によって、選択できる行動を制限する。詳細は上の説明参照。
    # 取引時間内 (CountDown >0)
    if(cd_ > 0):
      if(bool(lp_)):
        if(bool(sp_)):
          # パターン4：両建て(long_position = True, short_position = True)
          self.action_mask = [False, False, False, True, False, False]
        else:
          # パターン2：ロングポジション(long_position = True, short_position = False)
          self.action_mask = [True, False, False, True, True, True]
      else:
        if(bool(sp_)):
          # パターン3：ショートポジション(long_position = False, short_position = True)
          self.action_mask = [True, False, False, True, True, True]
        else:
          # パターン1：ポジションなし(long_position = False, short_position = False)
          self.action_mask = [True, True, True, False, False, False]
    # 取引時間外 (CountDown <= 0)
    else:
      if(bool(lp_)):
        if(bool(sp_)):
          # パターン8：両建て(long_position = True, short_position = True)
          self.action_mask = [False, False, False, True, False, False]
        else:
          # パターン6：ロングポジション(long_position = True, short_position = False)
          self.action_mask = [False, False, False, True, False, False]
      else:
        if(bool(sp_)):
          # パターン7：ショートポジション(long_position = False, short_position = True)
          self.action_mask = [False, False, False, True, False, False]
        else:
          # パターン5：ポジションなし(long_position = False, short_position = False)
          self.action_mask = [True, False, False, False, False, False]
    
    return self.action_mask
  
  def evaluateRewrd(self, action_index, data, position_open_price=None
                    , profit_loss=None, long_position=None, short_position=None):
    
    if(position_open_price is not None):
      self.position_open_price = position_open_price
    if(profit_loss is not None):
      self.profit_loss = profit_loss
    if(long_position is not None):
      self.long_position = long_position
    if(short_position is not None):
      self.short_position = short_position
    
    # 報酬をリセットする
    self.reward = 0.0
    # action_indexに従って、報酬とポジションを決定する
    # output_data(行動A[seq,6])
    # a0:何もしない
    # a1:LongEntry
    # a2:ShortEntry
    # a3:PositionClose
    # a4:PositionClose&LongEntry
    # a5:PositionClose&ShortEntry
    no_action = 0
    long_entry = 1
    short_entry = 2
    position_close = 3
    close_and_long = 4
    close_and_short = 5
    # print('Price Data Hour')
    # print(data)
    if(action_index[1] == no_action):
      self.reward = self.profit_loss
      # print('## No Action')
      pass
    
    elif(action_index[1] == long_entry):
      self.reward = self.profit_loss
      self.long_position = True
      self.position_open_price = float(data[6])
      # print('## Long Entry')
    
    elif(action_index[1] == short_entry):
      self.reward = self.profit_loss
      self.short_position = True
      self.position_open_price = float(data[6])
      # print('## Short Entry')
    
    elif(action_index[1] == position_close):
      #Rewardの計算
      self.reward = self.profit_loss
      #クローズしたときの損益の状況によりボーナスと罰を与える
      # if(self.profit_loss - MARGIN > 0.0):
      if(self.profit_loss > 0.0):
        self.reward += 1.0
      else:
        self.reward += -1.0
      # self.total_profit_loss += (self.profit_loss - MARGIN)
      # self.total_profit_loss += self.profit_loss
      self.total_profit_loss = np.append(self.total_profit_loss, self.profit_loss)
      # print('### Position Closed.')
      self.position_open_price = 0.0
      self.profit_loss = 0.0
      self.long_position = False
      self.short_position = False
    
    elif(action_index[1] == close_and_long):
      #Rewardの計算
      self.reward = self.profit_loss
      #クローズしたときの損益の状況によりボーナスと罰を与える
      # if(self.profit_loss - MARGIN > 0.0):
      if(self.profit_loss > 0.0):
        self.reward += 1.0
      else:
        self.reward += -1.0
      # self.total_profit_loss += (self.profit_loss - MARGIN)
      # self.total_profit_loss += self.profit_loss
      self.total_profit_loss = np.append(self.total_profit_loss, self.profit_loss)
      # print('### Position Closed. Long Entry')
      self.position_open_price = float(data[6])
      self.profit_loss = 0.0
      self.long_position = True
      self.short_position = False
    
    elif(action_index[1] == close_and_short):
      #Rewardの計算
      self.reward = self.profit_loss
      #クローズしたときの損益の状況によりボーナスと罰を与える
      # if(self.profit_loss - MARGIN > 0.0):
      if(self.profit_loss > 0.0):
        self.reward += 1.0
      else:
        self.reward += -1.0
      # self.total_profit_loss += (self.profit_loss - MARGIN)
      # self.total_profit_loss += self.profit_loss
      self.total_profit_loss = np.append(self.total_profit_loss, self.profit_loss)
      # print('### Position Closed. Short Entry')
      self.position_open_price = float(data[6])
      self.profit_loss = 0.0
      self.long_position = False
      self.short_position = True
    
    else:
      print('### ERROR IN TAKE ACTION.')
    
    # print(' Reward:{:.4f}, TotalProfitLoss:{:.4f}, PositionOpenPrice:{:.4f}, ProfitLoss:{:.4f}, LongPosition:{:.4f}, ShortPosition:{:.4f}'
    #        .format(self.reward, np.sum(self.total_profit_loss), self.position_open_price, self.profit_loss, self.long_position, self.short_position))

    return
    
  def setProfitLoss(self, pl):
    self.profit_loss = pl
    return
    
  def setLongPosition(self, lp):
    self.long_position = lp
    return
    
  def setShortPosition(self, sp):
    self.short_position = sp
    return
    
  def setAccountStatus(self, acs):
    self.setProfitLoss(acs[0])
    self.setLongPosition(acs[1])
    self.setShortPosition(acs[2])
    return

  def getAccountStatus(self):
    return (self.total_profit_loss, self.profit_loss, self.long_position, self.short_position)
  
  def getReward(self):
    return self.reward
  
  def getNextPeriod(self, yr, wk, wd, hr):
    # トレーニングをする次の期間を返す
    # 原則として、より大きい時間単位がNoneは受け付けない(例：時間(hour)に値が入っていて、曜日(weekday)に値がない)
    if(hr is None):
      if(wd is None):
        if(wk is None):
          if(yr is None):
            print('■■ERROR in getNextPeriod() All period are None.')
          else:
            # 年データのみの場合は翌年までシフトさせる
            self.next_period = datetime(yr,1,1)+timedelta(days=366)
            return (self.next_period.year, None, None, None)
        else:
          # 週までの場合は1週間シフトさせる
          self.next_period = datetime.strptime(str(yr) + '-' + str(wk) + '-1','%G-%V-%u')+timedelta(weeks=1)
          return (self.next_period.year, self.next_period.isocalendar()[1], None, None)
      else:
        # 曜日がある場合は、翌日にシフトさせる。ただし、金曜日(5)の時は3日シフトさせる
        self.current_period = datetime.strptime(str(yr) + '-' + str(wk) + '-' + str(wd),'%G-%V-%u')
        if(self.current_period.isoweekday() == 5):
          self.next_period = self.current_period + timedelta(days=3)
        elif(self.current_period.isoweekday() == 6):
          # 土曜日はあり得ないが例外処理
          print('■■ERROR in getNextPeriod() Weekday is set 6(Saturday).')
          self.next_period = self.current_period + timedelta(days=2)
        else:
          self.next_period = self.current_period + timedelta(days=1)
        return (self.next_period.year, self.next_period.isocalendar()[1], self.next_period.isocalendar()[2], None)
    else:
      # 時間があるときは1時間シフトさせる。ただし、金曜日の23時の時は2日と1時間シフトさせる
      self.current_period = datetime.strptime(str(yr) + '-' + str(wk) + '-' + str(wd) + '-' + str(hr),'%G-%V-%u-%H')
      if(self.current_period.isoweekday() == 5 and self.current_period.hour == 23):
        self.next_period = self.current_period + timedelta(days=2,hours=1)
      elif(self.current_period.isoweekday() == 6):
        # 土曜日はあり得ないが例外処理
        print('■■ERROR in getNextPeriod() Weekday is set 6(Saturday)_2.')
        self.next_period = (self.current_period + timedelta(days=2)).replace(hour=0)
      elif(self.current_period.isoweekday() == 7):
        # 日曜日はあり得ないが例外処理
        print('■■ERROR in getNextPeriod() Weekday is set 7(Sunday).')
        self.next_period = (self.current_period + timedelta(days=1)).replace(hour=0)
      else:
        self.next_period = self.current_period + timedelta(hours=1)
      return (self.next_period.year, self.next_period.isocalendar()[1], self.next_period.isocalendar()[2], self.next_period.hour)
    

  

##Class Agent
・LSTMのニューラルネットを構築する<br>
・与えられた状態から行動を決定する<br>
・次の行動から得られた報酬とその時の状態からその次の行動関数を算出し、誤差を補正する<br>

###input_data(状態S[seq,8])
s0 = 現在の損益(pips/1000) 例えば101.000USDJPY - 100.000USDJPY=1000pips<br>
s1 = LongPosition (0=none, 1=entry)<br>
s2 = ShortPosition (0=none, 1=entry)<br>
s3 = Weekday(0/6 = Monday, 6/6 = Sunday)<br>
s4 = Countdown：毎週月曜日の0:00足を1、金曜日の22:00足を0となるようにする。金曜日の23:00足はマイナスの値となる。<br>
s5 = Close-Open<br>
s6 = High-Low<br>
s7 = Tickvol(1/100000)<br>
※今後、複数symbolをインプットとする場合は、s5-s7のセットを追加する。

###output_data(行動A[seq,6])
a0:何もしない<br>
a1:LongEntry<br>
a2:ShortEntry<br>
a3:PositionClose<br>
a4:PositionClose&LongEntry<br>
a5:PositionClose&ShortEntry<br>
※両建てはしない。ドテンを想定する。

    
    
 



In [0]:
#@title class Agent { output-height: 200 }
hidden_num = 8 #@param {type:"integer"}

class Agent():
  
  def __init__(self,input_num,output_num,LoadModel=False, ModelName=None, FilePath=None): 
    #引数の各素子数に応じたニューラルネットを作成する
    self.brain = Brain(input_num,output_num,hidden_num)
    
    #これまでのトレーニングパラメータを呼び出す
    if(LoadModel):
      if(FilePath is not None):
        ModelName = FilePath + ModelName
      self.brain.loadModel(ModelName)
    
  def prepareModel(self,input_data):
    #初めに1週間分（想定）の時系列データをニューラルネットに読み込ませ
    #cell_stateに記憶させる
    #この時の戻り値（action_score[●(seq_num),input_num]
    # self.hidden_cell_state = self.brain.getHiddenCellState()
    with torch.no_grad():
      #self.action_score = self.brain(input_data, self.hidden_cell_state)
      self.action_score = self.brain(input_data)
      # 疑似的にprev_action_scoreにも値を入れる
      #self.prev_action_score = self.action_score[-1].view(1,-1)
      # ひとまず、Noneを格納する
      self.prev_action_score = None
      #疑似的にrewardにも値を入れる
      self.reward = 0.0
    
    return self.action_score
    
  def getActionScore(self, input_data):
    #1本ずつの時間足を取得して、出力を得る
    # print(input_data)
    #バックプロパゲーションをするためには、Variable()で包む?
    #self.hidden_cell_state = self.brain.getHiddenCellState()
    #self.action_score = self.brain(input_data, self.hidden_cell_state)
    self.action_score = self.brain(input_data)
    return self.action_score
  
  def stackLossData(self
                    , prev_action_score=None
                    , action_score=None
                    , reward=None):
    if(prev_action_score is not None):
      self.prev_action_score = prev_action_score
    if(action_score is not None):
      self.action_score = action_score
    if(reward is not None):
      self.reward = reward
    self.brain.stackLossData(self.prev_action_score, self.action_score, self.reward)
    return

  def evaluateLossFunction(self):
    self.brain.evaluateLossFunction()
    return
    
  def setPrevActionScore(self, acs_=None):
    if(acs_ is not None):
      self.prev_action_score = acs_
    else:
      self.prev_action_score = self.action_score
    return
    
  def getMaxActionIndex(self, action_mask,episode_num=-2,action_score=None):
    if(action_score is not None):
      self.action_score = action_score
    # print('action_score')
    # print(self.action_score[0])
    # print('action_mask')
    # print(action_mask)
    self.action_index = self.brain.decideAction(action_mask,self.action_score[0],episode_num)
    return self.action_index
  
  def getHiddenCellState(self):
    return self.brain.getHiddenCellState()
  
  def updateNN(self):
    self.brain.updateNN()
    
  def resetHiddenCellState(self):
    self.brain.resetHiddenCellState()
    return
  
  def clearLossData(self):
    self.brain.clearLossData()
    return
  
  def saveModel(self, SaveModel,ModelName, FilePath=None):
    if (SaveModel):
      if(FilePath is not None):
        ModelName = FilePath + ModelName

      self.brain.saveModel(ModelName)
    
    return
  

##Class Brain
Agent内のニューラルネット部分をBrainクラスとして別途定義

In [0]:
#@title class Brain { output-height: 200 }
GAMMA = 0.99

class Brain(nn.Module):

  def __init__(self,input_num,output_num,hidden_num):
    self.input_num = input_num
    self.output_num = output_num
    self.hidden_num = hidden_num

    super(Brain, self).__init__()
    #引数の各素子数に応じたニューラルネットを作成する
    #モデルとしては入力層(input_num)→LSTM→LSTM出力層（=hidden_num)→出力層(output_num)
    self.lstm = nn.LSTM(input_num, hidden_num)
    self.lstm2action = nn.Linear(hidden_num, output_num)

    #hidden_stateとcell_stateを初期化する。hidden_cell_stateは(hidden_state,cell_state)のタプルにする
    #それぞれのテンソルのサイズは(num_layers * num_directions, batch, hidden_size)となる。
    #self.hidden_state = torch.randn(1, 1, hidden_num)
    #self.cell_state = torch.randn(1, 1, hidden_num)
    #初期化はzreros()で行う。
    self.resetHiddenCellState()
    
    #損失関数と最適化手法を定義する
    #損失関数は、（ひとまず）mooth_l1_loss(state_action_values,expected_state_action_values)を採用
    #optimiserは、optim.Adam(self.model.parameters(), lr=0.0001)
    self.optimizer = optim.Adam(self.parameters(), lr=0.0001)
    #self.loss_function = F.smooth_l1_loss()
    #self.loss = 0.0
    self.criterion = nn.SmoothL1Loss()
    return

  
  #def forward(self,input_data,hidden_cell_state):
  def forward(self,input_data):
    #状態は毎時間足ごとに変わるため、逐次更新法とする(要検討)
    #input_data(●(seq_num),1(batch_size),input_num)
    #hidden_state(1,1,hidden_num)
    #cell_state(1,1,hidden_num)
    #output_data(=action_score)(●(batchi_size),output_num)
    
    #引数のNumpyをtorch.tensorに変換
    self.input_data = torch.FloatTensor(input_data)
    self.lstm_out, (self.hidden_state, self.cell_state) = self.lstm(self.input_data.view(-1, 1, self.input_num)
                                                                    ,(self.hidden_state, self.cell_state))
    self.action_score = self.lstm2action(self.lstm_out.view(-1, self.hidden_num))
    return self.action_score
  
  def clearLossData(self):
    #損失関数に渡すデータの中身を初期化する
    self.output = torch.empty(0,1,requires_grad=True)
    self.target = torch.empty(0,1,requires_grad=True)
    # print('clearLossData')
    # print('output')
    # print(self.output)
    # print('target')
    # print(self.target)
    
  # def evaluateLossFunction(self, prev_action_score, action_score, reward):
  def stackLossData(self, prev_action_score, action_score, reward):
    
    self.prev_action_score = prev_action_score
    self.action_score = action_score
    self.reward = reward
    # prev_action_scoreに値が入っていない1回目は処理を飛ばす
    if( self.prev_action_score is None):
      # print('prev_action_score is None in evaluateLossFunction()')
      return
    
    # print('# prev_action_score in evaluateLossFunction()')
    # print(self.prev_action_score)
    # print('# action_score in evaluateLossFunction()')
    # print(self.action_score)
    # print('# reward in evaluateLossFunction()')
    # print(self.reward)
    
    # print('before loss = {:.4f}'.format(self.loss))
    
    self.expected_action_values = self.reward + GAMMA*self.action_score[0].max(0)[0]
    # self.loss = F.smooth_l1_loss(Variable(torch.tensor(prev_action_score[0].max(0)[0]))
    #                              , Variable(torch.tensor(self.expected_action_values)))
    # self.loss = self.criterion(torch.tensor(self.prev_action_score[0].max(0)[0], requires_grad=True)
    #                           , torch.tensor(self.expected_action_values, requires_grad=True))
    self.output = torch.cat([self.output, self.prev_action_score[0].max(0)[0].clone().detach().requires_grad_(True).view(-1,1)], dim=0)
    self.target = torch.cat([self.target, self.expected_action_values.clone().detach().requires_grad_(True).view(-1,1)], dim=0)
    # print('output')
    # print (self.output)
    # print('target')
    # print(self.target)

  def evaluateLossFunction(self):
    # self.loss = self.criterion(self.prev_action_score[0].max(0)[0].clone().detach().requires_grad_(True).view(-1,1)
    #                            , self.expected_action_values.clone().detach().requires_grad_(True).view(-1,1))
    self.loss = self.criterion(self.output, self.target)
    # print('# output in evaluateLossFunction()')
    # print(self.output)
    # print('# target in evaluateLossFunction()')
    # print(self.target)
    # print('# loss in evaluateLossFunction(self)')
    # print(self.loss)
    # print('prev_action_values')
    # print(prev_action_score[0].max(0)[0])
    # print('expected_action_values')
    # print(self.expected_action_values)
    
    # print('after loss = {:.4f}'.format(self.loss))
    # for debug
    # self.loss = 0.0
    return
    
  def resetHiddenCellState(self): 
    self.hidden_state = torch.zeros(1, 1, self.hidden_num)
    self.cell_state = torch.zeros(1, 1, self.hidden_num)
    
  def decideAction(self, action_mask, action_score,episode=-2):
    #取得したaction_score[1,output_num]にその時に可能な行動をaction_mask[output_num]によりマスクし、
    #その中で最大の行動（インデックス）を返す
    # ε-greedy法を適用する
    # epsilon = 0.5 * (1 / (episode + 1))
    epsilon = 1 / (episode + 1)
    
    if epsilon <= np.random.uniform(0, 1):
      self.action_index = torch.where(torch.ByteTensor(action_mask),action_score
                                      ,torch.full_like(action_score,float('-inf'))).max(0)
      return self.action_index
    else:
      # print('==Action Random Select==')
      self.action_index = torch.where(torch.ByteTensor(action_mask),torch.rand_like(action_score)
                                      ,torch.full_like(action_score,float('-inf'))).max(0)
      return self.action_index

  def updateNN(self):
    # if(self.prev_action_score is None):
      # print('prev_action_score is None in updateNN()')
      # return
    
    # ネットワークを更新します
    self.optimizer.zero_grad()  # 勾配をリセット
    self.loss.backward()  # バックプロパゲーションを計算
    self.optimizer.step()  # 結合パラメータを更新
    return
  
  def setHiddenCellState(self, hidden_cell_state):
    (self.hidden_state, self.cell_state) = hidden_cell_state
    return
  
  def getHiddenCellState(self):
    return (self.hidden_state, self.cell_state)
  
  def saveModel(self, PATH):
    torch.save(self.state_dict(), PATH)
    return
  
  def loadModel(self, PATH):
    self.load_state_dict(torch.load(PATH))
    return
  

##Class Main

In [0]:
#@title class Main { output-height: 200 }
EPISODE_NUM = 1000 #@param {type:"integer"}

action_num = 6 #@param {type:"integer"}
state_num = 8 #@param {type:"integer"}

SYMBOL = "EURUSD" #@param ["EURUSD", "USDJPY","GBPUSD","EURJPY","EURGBP","GBPJPY"]
PERIOD = "H1" #@param ["H1", "M1"]
FILE_PATH_PRICE = r"/content/drive/My Drive/Colab Notebooks/PriceData/"

YEAR = 2018
WEEKNUM = 31 #@param {type:"slider", min:1, max:52, step:1}
WEEKDAY = None #@param {type:"slider", min:1, max:5, step:1}
HOUR = None #@param {type:"slider", min:0, max:23, step:1}
SAVE_MODEL = True #@param {type:"boolean"}
LOAD_MODEL = True #@param {type:"boolean"}

ModelName = 'Model_' + SYMBOL + '_' + PERIOD + '.pth'
ModelPath = r"/content/drive/My Drive/Colab Notebooks/ModelParameter/"

result_df = pd.DataFrame(np.empty((0,0)))

#トレード環境を初期化する。
TradeEnv=Environment(SYMBOL,PERIOD)
TradeEnv.makePriceData(SYMBOL,PERIOD,FILE_PATH_PRICE)

# トレードエージェントを初期化する。
TradeAgt = Agent(state_num,action_num,LOAD_MODEL,ModelName,ModelPath)
hidden_cell_state = TradeAgt.getHiddenCellState()
# print('hidden_cell_state_0')
# print(hidden_cell_state)

# 損益結果書き出し用ファイルの準備
FILE_PATH =  r"/content/drive/My Drive/Colab Notebooks/Result/"
FILE_NAME = 'TotalProfitLoss.csv'

#pandasのto_csv()を利用する
# with open(FILE_NAME, mode='w') as f:
#   header = '<Episode>,<TotalProfitLoss>\n'
#   f.write(header)

#エピソードを繰り返す場合はここから繰り返す
#WEEKNUMは、2-50の範囲でランダムに発生させる
for ep_num in range(EPISODE_NUM):
  # hidden_cell_stateTD誤差のミニバッチを初期化する
  TradeAgt.resetHiddenCellState()
  TradeAgt.clearLossData()
  # AccountStatusを初期化する
  TradeEnv.clearAccountState()
  
  # 任意の連続するWEEKNUMをランダムに取得する
  # WEEKNUM = random.randrange(2,50,1)
  year = YEAR
  week_num = WEEKNUM
  week_day = WEEKDAY
  hour_num = HOUR
  pricedata_week = TradeEnv.getPriceData(year,week_num,week_day,hour_num)
  inputdata_week = TradeEnv.getInitialPriceData(pricedata_week)
  TradeEnv.clearAccountState()

  #print(inputdata_week)
  '''
  FILE_PATH = ''
  FILE_NAME = 'input_data_week.csv'
  with open(FILE_NAME, mode='w') as f:
    for row in inputdata_week:
      for e in row:
        f.write(str(e)+',')
      f.write('\n')
  '''

  # (hidden_state, cell_state) = TradeAgt.getHiddenCellState()
  #はじめの1週間のプライスデータをニューラルネットに読み込ませる
  action_score = TradeAgt.prepareModel(inputdata_week)
  # hidden_cell_state = TradeAgt.getHiddenCellState()
  # print(action_score)
  # print('hidden_cell_state_1')
  # print(hidden_cell_state)


  # 初期化した次の期間のプライスデータを使ってトレーニングを行う
  # HOURが指定されている場合は、HOURを1時間進める
  (year,week_num,week_day,hour_num) = TradeEnv.getNextPeriod(
      year,week_num,week_day,hour_num)
  
  '''
  if(hour_num is not None):
    if(hour_num == 23):
      if(week_day == 4):
        week_num += 1
        week_day = 0
      else:
        week_day += 1
      hour_num = 0
    else:
      hour_num += 1
  elif(week_day is not None):
    if(week_day == 4):
      week_num += 1
      week_day = 0
    else:
      week_day += 1
  elif(week_num is not None):
    week_num += 1
  '''
  
  pricedata_next_week = TradeEnv.getPriceData(year,week_num,week_day,hour_num)
  # inputdata_next_week = TradeEnv.getInitialPriceData(pricedata_next_week)

  for pricedata_hour in pricedata_next_week:
    #pricedata_hourに現在のポジションの状況を付加したinputdata_hourを取得する
    inputdata_hour = TradeEnv.getInitialPriceData(pricedata_hour.reshape([1,pricedata_hour.shape[0]]))[0]
    # print('# inputdata_hour in main()')
    # print(inputdata_hour)
    #現在の状況から、選択できる行動を受け取る
    action_mask = TradeEnv.getAvailableAction(*inputdata_hour[0:3],inputdata_hour[4])

    action_score = TradeAgt.getActionScore(inputdata_hour)
    reward = TradeEnv.getReward()
    # 新しく得たaction_scoreを1つ前のprev_action_score(TradeAgtに記憶されている)と報酬(TradeEnvにある)から損失関数の
    # 入力データをミニバッチにストックする
    # TradeAgt.evaluateLossFunction(reward=reward)
    TradeAgt.stackLossData(reward=reward)
    # action_scoreとaction_maskを基に、次の行動を決定する。
    action_index = TradeAgt.getMaxActionIndex(action_mask,ep_num)
    #action_index = TradeAgt.getMaxActionIndex(action_mask)
    
    # print('action_index')
    # print(action_index)
    # 選択したアクションに従い報酬、オープンプライス、ポジションを更新する
    TradeEnv.evaluateRewrd(action_index, pricedata_hour)
    # TradeAgt.updateNN()

    # TradeAgtに記憶されているaction_scoreをprev_action_scoreにずらす
    TradeAgt.setPrevActionScore()

  #1エピソード終わったら、ニューラルネットを更新する
  TradeAgt.evaluateLossFunction()
  TradeAgt.updateNN()

  Account_Status = TradeEnv.getAccountStatus()
  # print('■■Total_Profit_Loss■■')
  df = pd.DataFrame(pd.Series(Account_Status[0]).describe()).T
  df['sum'] = Account_Status[0].sum()
  # print(df)
                    
  # print('[ep' + str(ep_num) + '] ' + str(Account_Status[0]))
  
  # 損益結果追記
  # pandas の to_csv()を利用する
  # with open(FILE_NAME, mode='a') as f:
    # result = str(ep_num) + ',' + str(np.sum(Account_Status[0])) + '\n'
  #   result = str(df.to_numpy()) + '\n'
  #   f.write(result)
  
  # 損益結果追記
  result_df = result_df.append(df,ignore_index=True)

# テストが完了したニューラルネットを保存する
TradeAgt.saveModel(SAVE_MODEL,ModelName,ModelPath)

print('◆◆Test finished.◆◆')
result_df.to_csv(FILE_PATH+FILE_NAME)
result_df


◆◆Test finished.◆◆


,count,mean,std,min,25%,50%,75%,max,sum
0,55.0,0.021309,0.137339,-0.237,-0.04700,0.0100,0.07450,0.632,1.172
1,79.0,-0.021835,0.093625,-0.426,-0.05000,-0.0100,0.02550,0.208,-1.725
2,86.0,-0.029163,0.103673,-0.490,-0.05550,-0.0140,0.02550,0.231,-2.508
3,53.0,-0.041377,0.122281,-0.490,-0.06500,-0.0160,0.02700,0.134,-2.193
4,78.0,-0.026833,0.109594,-0.490,-0.05900,-0.0115,0.02475,0.231,-2.093
5,81.0,-0.016185,0.103469,-0.490,-0.04000,-0.0020,0.03400,0.231,-1.311
6,95.0,-0.016853,0.108855,-0.355,-0.05500,-0.0130,0.03100,0.521,-1.601
7,97.0,-0.016856,0.109869,-0.425,-0.05400,-0.0100,0.02700,0.521,-1.635
8,77.0,-0.008766,0.104824,-0.490,-0.03600,-0.0010,0.03400,0.231,-0.675
9,108.0,-0.020204,0.099118,-0.490,-0.04950,-0.0045,0.02700,0.231,-2.182


In [0]:
SYMBOL = "USDJPY" #@param ["EURUSD", "USDJPY","GBPUSD","EURJPY","EURGBP","GBPJPY"]
PERIOD = "M1" #@param ["H1", "M1"]

WEEKNUM = 18 #@param {type:"slider", min:1, max:52, step:1}
SAVE_MODEL = False #@param {type:"boolean"}
LOAD_MODEL = False #@param {type:"boolean"}

ModelName = 'Model_' + SYMBOL + '_' + PERIOD + '.pth'
ModelPath = None

result_df = pd.DataFrame(np.empty((0,0)))

#トレード環境を初期化する。
TradeEnv=Environment()
TradeEnv.makePriceData(SYMBOL,PERIOD)
TradeEnv.price.shape

[['DateTime' 'WeekNum' 'Weekday' 'Open' 'High' 'Low' 'Close' 'Tickvol']
 [datetime.datetime(2019, 3, 4, 0, 2) 10 0 '111.852' '111.855' '111.852'
  '111.852' '3']
 [datetime.datetime(2019, 3, 4, 0, 3) 10 0 '111.852' '111.852' '111.849'
  '111.852' '4']
 [datetime.datetime(2019, 3, 4, 0, 4) 10 0 '111.852' '111.853' '111.852'
  '111.853' '2']
 [datetime.datetime(2019, 3, 4, 0, 5) 10 0 '111.852' '111.860' '111.852'
  '111.860' '11']
 [datetime.datetime(2019, 3, 4, 0, 6) 10 0 '111.860' '111.861' '111.852'
  '111.852' '46']
 [datetime.datetime(2019, 3, 4, 0, 7) 10 0 '111.851' '111.856' '111.811'
  '111.817' '60']
 [datetime.datetime(2019, 3, 4, 0, 8) 10 0 '111.816' '111.825' '111.816'
  '111.816' '225']
 [datetime.datetime(2019, 3, 4, 0, 9) 10 0 '111.818' '111.836' '111.816'
  '111.824' '61']
 [datetime.datetime(2019, 3, 4, 0, 10) 10 0 '111.827' '111.838' '111.825'
  '111.838' '41']]


(64582, 8)

In [0]:
weekprice = TradeEnv.price[TradeEnv.price[:,1]==18]
dayprice = weekprice[weekprice[:,2] == 3]
hourprice = [d for d in dayprice if d[0].hour == 21]

array([False, False, False, ...,  True,  True,  True])

In [0]:
weekprice = TradeEnv.price[TradeEnv.price[:,1]==18]
dayprice = weekprice[weekprice[:,2] == 3]
hourprice =np.array([d for d in dayprice if d[0].hour == 21])
hourprice.shape


(60, 8)

In [0]:
TradeEnv.extract_price[0:10]

array([list([]),
       list([datetime.datetime(2018, 1, 2, 0, 0), 1, 2, '112.503', '112.663', '112.503', '112.586', '827']),
       list([datetime.datetime(2018, 1, 2, 1, 0), 1, 2, '112.591', '112.779', '112.588', '112.764', '2342']),
       list([datetime.datetime(2018, 1, 2, 2, 0), 1, 2, '112.761', '112.784', '112.714', '112.728', '1082']),
       list([datetime.datetime(2018, 1, 2, 3, 0), 1, 2, '112.727', '112.742', '112.651', '112.713', '1136']),
       list([datetime.datetime(2018, 1, 2, 4, 0), 1, 2, '112.712', '112.746', '112.697', '112.709', '2282']),
       list([datetime.datetime(2018, 1, 2, 5, 0), 1, 2, '112.708', '112.714', '112.660', '112.696', '1512']),
       list([datetime.datetime(2018, 1, 2, 6, 0), 1, 2, '112.696', '112.725', '112.688', '112.702', '1935']),
       list([datetime.datetime(2018, 1, 2, 7, 0), 1, 2, '112.704', '112.733', '112.675', '112.685', '2279']),
       list([datetime.datetime(2018, 1, 2, 8, 0), 1, 2, '112.686', '112.699', '112.645', '112.665', '441